Dependencies and dataset

In [1]:
import numpy as np
import pandas as pd # only for reading csv
import zipfile
import os

In [2]:
!kaggle datasets download -d oddrationale/mnist-in-csv -p ./data
with zipfile.ZipFile('./data/mnist-in-csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')
os.remove('./data/mnist-in-csv.zip')


  0%|          | 0.00/15.2M [00:00<?, ?B/s]
  7%|▋         | 1.00M/15.2M [00:00<00:09, 1.60MB/s]
 20%|█▉        | 3.00M/15.2M [00:00<00:02, 4.61MB/s]
 46%|████▌     | 7.00M/15.2M [00:00<00:00, 11.3MB/s]
 72%|███████▏  | 11.0M/15.2M [00:01<00:00, 16.6MB/s]
 98%|█████████▊| 15.0M/15.2M [00:01<00:00, 21.7MB/s]
100%|██████████| 15.2M/15.2M [00:01<00:00, 13.6MB/s]


Define cost and activation functions

In [3]:
"""Activation functions and derivatives with respect to z"""
def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

def ReLu(z):
    return np.maximum(0,z)

def ReLu_derivative(z):
    return np.where(z > 0, 1.0, 0.001)

"""Cost functions and derivatives with respect to a"""
def cross_entropy(y,a):
    return np.sum(np.nan_to_num(-y*np.log(a) - (1-y)*np.log(1-a)))

def cross_entropy_derivative(y,a):
    return np.nan_to_num((a-y) / (a*(1-a)))

def quadratic(y,a):
    return np.sum(0.5*(y-a)**2)

def quadratic_derivative(y,a):
    return a-y

activation_functions = {'sigmoid': (sigmoid, sigmoid_derivative), 'relu': (ReLu, ReLu_derivative)}
cost_functions = {'quadratic': (quadratic, quadratic_derivative), 'cross_entropy': (cross_entropy, cross_entropy_derivative)}

Define Network class

In [4]:
class Network:
    def __init__(self, layers, cost_function = 'cross_entropy', activation_function = 'sigmoid'):
        self.L = layers
        self.W = [np.random.randn(x,y)/np.sqrt(y) for x,y in zip(self.L[1:],self.L[0:-1])] # divide stadard deviation to avoid saturation
        self.B = [np.random.randn(x,1) for x in self.L[1:]]

        if cost_function in cost_functions:
            self.cost_function, self.cost_function_derivative = cost_functions[cost_function]
        else:
            raise ValueError(f"Invalid cost function: {cost_function}")
        if activation_function in activation_functions:
            self.activation_function, self.activation_derivative = activation_functions[activation_function]
        else:
            raise ValueError(f"Invalid activation function: {activation_function}")

    def feedforward(self, X):
        A = X
        for w, b in zip(self.W, self.B):
            Z = np.dot(w, A) + b
            A = self.activation_function(Z)
        return A

    def fit(self, train_set, batch_size, epochs, eta, lmbda, patience=10, valid_set=None):
        # Set-up
        X_train, Y_train = train_set[0], train_set[1]
        if valid_set is not None:
            X_valid, Y_valid = valid_set[0], valid_set[1]
            print("Tracking progress on the validation set:")
        else:
            print("Tracking progress on the training set:")
        best_acc, lowest_cost, no_progress_count = 0.0, np.Inf, 0

        # Training     
        for epoch in range(epochs):
            X_batches = np.array_split(X_train, X_train.shape[1] // batch_size, axis=1)
            Y_batches = np.array_split(Y_train, Y_train.shape[1] // batch_size, axis=1)
            for X_batch, Y_batch in zip(X_batches, Y_batches):  
                nabla_B = [np.zeros(b.shape) for b in self.B]
                nabla_W = [np.zeros(w.shape) for w in self.W]
                for i in range(X_batch.shape[1]):
                    a = X_batch[:,i].reshape(-1,1)
                    y = Y_batch[:,i].reshape(-1,1)
                    W_shifts, B_shifts = self.get_shifts(a, y)
                    nabla_B = [nb+dnb for nb, dnb in zip(nabla_B, B_shifts)] 
                    nabla_W = [nw+dnw for nw, dnw in zip(nabla_W, W_shifts)]
                self.W = [w-eta*nw/X_batch.shape[1] - eta*lmbda*w/X_train.shape[1] for w, nw in zip(self.W, nabla_W)] # L2 regularization
                self.B = [b-eta*nb/X_batch.shape[1] for b, nb in zip(self.B, nabla_B)]

            # Tracking progress
            if valid_set is not  None:
                acc, cost = self.__track_progress(X_valid, Y_valid) 
            else:
                acc, cost = self.__track_progress(X_train, Y_train)

            if acc > best_acc or cost < lowest_cost:
                no_progress_count=0
                if acc > best_acc:
                    best_acc = acc
                else:
                    no_progress_count += 1
                if cost < lowest_cost:
                    lowest_cost = cost      
                else:
                    no_progress_count += 2
                best_W, best_B = self.W, self.B
            else:
                no_progress_count += 3
            if no_progress_count > patience:
                self.W, self.B = best_W, best_B
                print(f"Early stopping: no improvement on validation set for {patience} epochs. Saving parameters from epoch {epoch-patience}.")
                break
            else:    
                print(f"epoch: {epoch}, ACC: {acc}, cost: {cost}")
                
    def __track_progress(self, X, Y):
        """
        Evaluates accuracy and cost and the end of each epoch.
        """
        acc = self.evaluate(X, Y)[1]
        cost = self.cost_function(Y, self.feedforward(X))
        return acc, cost

    def get_shifts(self, a, y):
        """
        Updates network's weights and biases by applying SGD and backpropagation.
        """
        Z=[]
        A=[a]
        for w,b in zip(self.W,self.B):
            z = np.dot(w,A[-1])+b
            a=self.activation_function(z)
            Z.append(z)
            A.append(a)
        return self.__backprob(y, A, Z)

    def __backprob(self,y,A,Z):
        def delta(y,x,z):
            return self.cost_function_derivative(y,x)*self.activation_derivative(z)
            
        D = [delta(y,A[-1],Z[-1])] # eq. (1)
        for i in range(1,len(Z)):
            D.insert(0, np.dot(self.W[-i].T,D[0])*self.activation_derivative(Z[-i-1])) # eq. (2)
        B_shifts = D # eq. (3)
        W_shifts = []
        for a,d in zip(A[0:-1],D):
            W_shifts.append(np.dot(d,a.T)) # eq. (4)
        return W_shifts, B_shifts

    def evaluate(self, X_test, Y_test):
        correct_predictions = 0
        predictions = self.feedforward(X_test)
        for prediction, y in zip(predictions.T, Y_test.T):
            if np.argmax(prediction) == np.argmax(y):
                correct_predictions += 1
        return correct_predictions, correct_predictions/(X_test.shape[1])

Preprocess data

In [44]:
train = pd.read_csv('data\mnist_train.csv').to_numpy()
test = pd.read_csv('data\mnist_test.csv').to_numpy()

# Normalize data
X_train, Y_train = train[:,1:] / 255 , train[:,0]
X_test, Y_test = test[:,1:] / 255, test[:,0] 

# Shuffle training data
perm = np.random.permutation(len(X_train))
X_train = X_train[perm].T
Y_train = Y_train[perm]

# Divide to training, test and validation
X_valid = X_train[:,:10000]
X_train = X_train[:,10000:]
X_test=X_test.T

# Convert labels to one-hot encoded vectors
# Example: 2 -> (0,0,1,0,0,0,0,0,0,0) 
Y_valid = Y_train[:10000]
Y_train = Y_train[10000:]
Y_train = np.eye(10)[Y_train].T
Y_valid = np.eye(10)[Y_valid].T
Y_test = np.eye(10)[Y_test].T

Set up network

In [45]:
net = Network([784,30,10], cost_function='cross_entropy', activation_function='sigmoid')
train = [X_train, Y_train]
valid = [X_valid, Y_valid]

Start training

In [46]:
net.fit(train_set=train, batch_size=10, epochs=200, eta=0.1, lmbda=5.0, patience=10, valid_set=valid)

Tracking progress on the validation set:
epoch: 0, ACC: 0.9188, cost: 5410.0647749628115
epoch: 1, ACC: 0.9344, cost: 4509.51900163403
epoch: 2, ACC: 0.9401, cost: 4108.828540293254
epoch: 3, ACC: 0.9457, cost: 3855.323878866133
epoch: 4, ACC: 0.9492, cost: 3672.585495175106
epoch: 5, ACC: 0.9505, cost: 3542.1775510942753
epoch: 6, ACC: 0.953, cost: 3447.5979567402837
epoch: 7, ACC: 0.9534, cost: 3376.7663727253394
epoch: 8, ACC: 0.9541, cost: 3321.449788199254
epoch: 9, ACC: 0.9553, cost: 3275.6341125728695
epoch: 10, ACC: 0.9561, cost: 3235.297266063156
epoch: 11, ACC: 0.9568, cost: 3198.79388253256
epoch: 12, ACC: 0.9569, cost: 3165.850955168115
epoch: 13, ACC: 0.9573, cost: 3136.293984138369
epoch: 14, ACC: 0.9578, cost: 3109.8177461562163
epoch: 15, ACC: 0.9581, cost: 3085.9961343052196
epoch: 16, ACC: 0.959, cost: 3064.3320386105465
epoch: 17, ACC: 0.9593, cost: 3044.391173531185
epoch: 18, ACC: 0.9597, cost: 3025.871597070542
epoch: 19, ACC: 0.96, cost: 3008.5841576154257
epoch:

Evaluate

In [ ]:
net.evaluate(X_test, Y_test)

(9637, 0.9637)